In [3]:
# Importing LIBRARIES
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import os
import numpy as np
import pandas as np
import matplotlib.pyplot as plt
%matplotlib inline

AttributeError: module 'keras.backend.tensorflow_backend' has no attribute '_is_tf_1'

In [ ]:
# Specifying the path of the data(train,test,validaton)
train = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/train'
test = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/test'
val = 'C:/Users/Kibet/Desktop/General/project/classifier/input/chest_xray/chest_xray/val'

In [ ]:
# Here we have backended keras to tensorflow ,so we go for channel_last ie to specify
#the channel value as the last dimension in shape of the input.
img_width,img_height= 150,150
input_shape = (img_width,img_height,3)

In [ ]:
# ImageDataGenerator-Generate batches of tensor image data with real-time data augmentation. 
#The data will be looped over (in batches).

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2
                                   ,horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
# Here we import images directly from Directory by using flow_from_directory method.
#flow_from_directory() automatically infers the labels from the directory structure of the folders containing images

train_generator = train_datagen.flow_from_directory(
    train,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    val,
    target_size=(img_width, img_height),
    batch_size=16,
    class_mode='binary')

In [ ]:
#creating our model
model=Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))

In [ ]:
#Gives the summary of the layers used to create our model 
model.summary()

In [ ]:
# Here we use RMSPROP optimizer and BINARY_CROSSENTROPY as loss function
model.compile(loss='binary_crossentropy',
optimizer='rmsprop',
metrics=['accuracy'])

In [ ]:
# We Fit the model here using fit_generator as we are dealing with large datasets

#Improving accuracy using callbacks 
from keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss', 
                             min_delta=0.05, 
                             patience=2, 
                             verbose=0, 
                             mode='auto', 
                             baseline=None, 
                             restore_best_weights=True
                            )

training_visualization=model.fit_generator(
    train_generator,
    steps_per_epoch=5217 // 16,
    callbacks=[early_stopping],
    epochs=15,
    validation_data=validation_generator,
    validation_steps=17 // 16)

In [ ]:
scores=model.evaluate(validation_generator, verbose=0)

In [ ]:
print('model val loss:', scores[0])
print('model val accuracy:', scores[1])